# The F-Test (Second Example)

### Intro and objectives


### In this lab you will learn:
1. examples of multiple regression models.
2. how to fit multiple regression models in Python.
3. How to conduct F-Tests.


## What I hope you'll get out of this lab
* The feeling that you'll "know where to start" when you need to fit a multiple regression model.
* Worked Examples of multiple regression models
* How to interpret the results obtained

In [2]:
!pip install wooldridge

import wooldridge as woo
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 4.4 MB/s 


# Example 1. Determinants of salaries in the major baseball league




#### In this case we fit a multiple linear models to predict salaries in terms of a series of factors including years in the league, games played per year, batting average, home runs and runs batted per year

$ log(salary)=\beta_0+\beta_1*years+\beta_2*gamesyr+\beta_3*bavg+\beta_4*hrunsyr+\beta_5*rbisyr+u $



### Using the data in MLB1 where n=353 individuals

In [3]:
salaries = woo.dataWoo('mlb1')


In [4]:
salaries.head()

,salary,teamsal,nl,years,games,atbats,runs,hits,doubles,triples,...,runsyr,percwhte,percblck,perchisp,blckpb,hispph,whtepw,blckph,hisppb,lsalary
0,6329213.0,38407380.0,1,12,1705,6705,1076,1939,320,67,...,89.666664,70.277969,18.844229,10.877804,0.000000,0.0,70.277969,0.000000,0.0,15.660686
1,3375000.0,38407380.0,1,8,918,3333,407,863,156,38,...,50.875000,70.277969,18.844229,10.877804,18.844229,0.0,0.000000,10.877804,0.0,15.031906
2,3100000.0,38407380.0,1,5,751,2807,370,840,148,18,...,74.000000,70.277969,18.844229,10.877804,0.000000,0.0,70.277969,0.000000,0.0,14.946913
3,2900000.0,38407380.0,1,8,1056,3337,405,816,143,18,...,50.625000,70.277969,18.844229,10.877804,0.000000,0.0,70.277969,0.000000,0.0,14.880221
4,1650000.0,38407380.0,1,12,1196,3603,437,928,19,16,...,36.416668,70.277969,18.844229,10.877804,18.844229,0.0,0.000000,10.877804,0.0,14.316286


In [5]:
salaries.describe()

,salary,teamsal,nl,years,games,atbats,runs,hits,doubles,triples,...,runsyr,percwhte,percblck,perchisp,blckpb,hispph,whtepw,blckph,hisppb,lsalary
count,3.530000e+02,3.530000e+02,353.000000,353.000000,353.000000,353.000000,353.000000,353.000000,353.000000,353.000000,...,353.000000,330.000000,330.000000,330.000000,330.000000,330.000000,330.000000,330.000000,330.000000,353.000000
mean,1.345672e+06,3.079483e+07,0.475921,6.325779,648.424929,2168.592068,290.402266,584.736544,103.881020,16.733711,...,38.582988,72.631088,16.548926,10.819986,5.144886,2.004179,36.667635,2.931898,3.046493,13.492183
std,1.407352e+06,8.722411e+06,0.500129,3.880142,538.697499,2025.059766,301.010915,575.378562,104.323511,21.533336,...,23.862686,15.227258,13.668326,9.387961,11.109446,5.858419,37.637047,6.667690,8.415198,1.182466
min,1.090000e+05,8.854000e+06,0.000000,1.000000,7.000000,7.000000,1.000000,1.000000,0.000000,0.000000,...,0.500000,20.296301,3.741786,0.540870,0.000000,0.000000,0.000000,0.000000,0.000000,11.599103
25%,2.536000e+05,2.455733e+07,0.000000,3.000000,230.000000,632.000000,73.000000,164.000000,26.000000,3.000000,...,18.600000,67.668961,8.007545,1.962080,0.000000,0.000000,0.000000,0.000000,0.000000,12.443514
50%,6.750000e+05,3.413650e+07,0.000000,6.000000,520.000000,1585.000000,191.000000,419.000000,70.000000,9.000000,...,35.333332,74.131294,14.453978,10.877804,0.000000,0.000000,20.296301,0.000000,0.000000,13.422468
75%,2.250000e+06,3.779200e+07,1.000000,9.000000,930.000000,3071.000000,407.000000,818.000000,147.000000,23.000000,...,55.888889,82.948860,18.755629,16.330647,7.986870,0.000000,73.642937,1.561949,0.000000,14.626441
max,6.329213e+06,4.286600e+07,1.000000,20.000000,2729.000000,10554.000000,1570.000000,3025.000000,634.000000,142.000000,...,105.142860,94.696266,73.960030,31.037498,73.960030,31.037498,94.696266,31.037498,73.960030,15.660686


In [6]:
type(salaries)

pandas.core.frame.DataFrame

In [7]:
# OLS regression:
reg = smf.ols(
    formula='np.log(salary) ~ years + gamesyr + bavg + hrunsyr + rbisyr',
    data=salaries)

In [8]:
# We fit the model
results = reg.fit()


In [9]:
results.params

Intercept    11.192418
years         0.068863
gamesyr       0.012552
bavg          0.000979
hrunsyr       0.014429
rbisyr        0.010766
dtype: float64

## Based on the previous we have fitted the following model:

$ log(salary)=11.19+0.68*years+0.0125*gamesyr+0.00097*bavg+0.14*hrunsyr+0.0107*rbisyr+u $


## F-Test of overall statistical significance

#### For every linear regression model we fit it is a good practice to conduct a F-test of overall statistical significance to determine if the model is useful

#### Essentially we want to test if all coefficients are different from zero.

#### This test corresponds to the following null hypothesis:

 $H_0: Β_1=Β_2=Β_3=Β_4=0=Β_5=0 $


#### The good news is that this F-test is carried out by default

In [11]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         np.log(salary)   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     117.1
Date:                Thu, 17 Nov 2022   Prob (F-statistic):           2.94e-72
Time:                        11:33:49   Log-Likelihood:                -385.11
No. Observations:                 353   AIC:                             782.2
Df Residuals:                     347   BIC:                             805.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.1924      0.289     38.752      0.0

### Based on the previous results:

#### The F-statistic is large (117.1) and its associated p-value is almost cero (2.94e-72).

#### Therefore we reject the null hypothesis that the model is not statistically significant

#### Therefore the model IS statistically significant (in the sense of having variables that do have an effect on the outcome)